# Introduction

The European Transaction Log (EUTL) is the backbone of the European Union Emissions Trading System (EUETS). It implements the transfer of emission allowances between parties active in the EUETS. The EUTL provides data on regulated installations, their emissions as well as transfers of allowances.  

Assuming that the database has sucessfully populated with the EUTL data (see 1_create_database.ipynb), this notebooks introduces the first steps of analyzing the data.

# Packages and options

In [1]:
from eutl_orm import DataAccessLayer    # data access layer to access the database
from eutl_orm import (Installation, Account, AccountHolder,
                      Country)          # classes for the basic building blocks of the EUTL
import numpy as np 
import pandas as pd

Database connection settings

In [2]:
connectionSettings = dict(
    user="eutlAdmin", 
    host="localhost", 
    db="eutl_orm", 
    passw="1234",
    port=5432
)

# Connect to database

Connect to database and get a session:

In [3]:
dal = DataAccessLayer(**connectionSettings)
session = dal.session

# Installations by registry

Suppose we want to get a list of a installations registered in a given country. There are two way of getting this list: (1) filter all installations by the registry and (2) selecting a registry and get all installations.

Starting with the first method, use the session to query all installations with the registry being equal to the country code:

In [4]:
registry = "GB"
inst = session.query(Installation).filter_by(registry_id=registry)
print("Number of installations registered: ", inst.count())

Number of installations registered:  2028


So there are 1519 installations registered in Spain. The second way to obtain these installations first selects the country Spain. Given the country object, we then query installations simply as a property of the object:

In [5]:
gb = session.query(Country).get(registry)
print(gb)
inst = gb.installations_in_registry
print("Number of installations registered: ", inst.count())

<Country('GB', 'United Kingdom')>
Number of installations registered:  2028


The second methods has the advantage that we can easily obtain other properties of the country. E.g, lets see how many accounts are registered in the registry and how many account holders registered with an address in the country.

In [6]:
print("Number of accounts registered: ", len(gb.accounts))
print("Number of account holder: ", len(gb.accountHolders))

Number of accounts registered:  4725
Number of account holder:  2157


# Analyze compliance

Given our list installations in the country, let's analyze compliance of a specific installation. We illustrative purpose let's choose an installation with a combustion activity, i.e., activity code 1 or 20.

First, filter the list of installations according to the activity code:

In [7]:
inst_combustion = gb.installations_in_registry.filter(Installation.activity_id.in_([1, 20]))
print("Number of combustion activities:", inst_combustion.count())

Number of combustion activities: 1218


Suppose, we want to find the "Drax" Coal power plant. To do so, further filter the collection of combustion plants in Spain using the installation name. To execute the query, let's use the one method that will throw an error if we do not find a result or more than one result (Use .all() for all results and .fist() to extract the first result. .first will return None if no result is selected.)

In [8]:
drax = inst_combustion.filter(Installation.name.like("%Drax%")).one()
print(drax.id + ": " + drax.name)
print(drax.address)

GB_381: Drax Power Station
DRAX POWER LIMITED,
DRAX POWER STATION, PO BOX 3
YO8 8PQ SELBY, NORTH YORKSHIRE
United Kingdom


Alternatively, we could also use www.EUETS.INFO/installations and the search field there which provides the installation identifier GB_381:

In [9]:
drax = session.query(Installation).get("GB_381")
print(drax.id + ": " + drax.name)
print(drax.address)

GB_381: Drax Power Station
DRAX POWER LIMITED,
DRAX POWER STATION, PO BOX 3
YO8 8PQ SELBY, NORTH YORKSHIRE
United Kingdom


Compliance is implemented as a property on the installation. For convenience, .get_compliance() provides the compliance as pandas DataFrame.

In [10]:
drax.compliance
df_compliance = drax.get_compliance()
df_compliance.head(2)

,verified,allocatedTotal,allocatedFree,compliance_id,installation_id,year,surrendered,verifiedCummulative,allocated10c,allocatedNewEntrance,euetsPhase,surrenderedCummulative,verifiedUpdated,compliance
0,20771624.0,14554187.0,14554187.0,A,GB_381,2005,20771624.0,20771624.0,0.0,0.0,2005-2007,20771624.0,False,The number of allowances and ERUs/CERs surrend...
1,22764847.0,14554187.0,14554187.0,A,GB_381,2006,22764847.0,43536471.0,0.0,0.0,2005-2007,43536471.0,False,The number of allowances and ERUs/CERs surrend...


Lets see what kind of data are provided for the first year of the EUETS (2005):

In [11]:
print("Compliance 2005: ", df_compliance[df_compliance.year == 2005].T)
print("Allocation 2006: ", df_compliance[df_compliance.year == 2006].allocatedTotal)

Compliance 2005:                                                                          0
verified                                                      2.07716e+07
allocatedTotal                                                1.45542e+07
allocatedFree                                                 1.45542e+07
compliance_id                                                           A
installation_id                                                    GB_381
year                                                                 2005
surrendered                                                   2.07716e+07
verifiedCummulative                                           2.07716e+07
allocated10c                                                            0
allocatedNewEntrance                                                    0
euetsPhase                                                      2005-2007
surrenderedCummulative                                        2.07716e+07
verifiedUpdated     

In 2005 (and 2006), Drax received a free allocation of 14.5 million allowances and emitted 20.7 million tCO<sub>2</sub> The installation was compliant, i.e., the number of allowances surrendered equaled verified emissions.

What kind of units have been surrendered and where did they come from? Details are also implemented either as property of the installation and with a get method to obtain a dataframe.

In [12]:
drax.surrendering
df_surrender = drax.get_surrendering()
df_surrender

,project_id,amount,year,originatingRegistry_id,unitType_id,installation_id,unitType
0,None,14000,2005,AT,EUA2005,GB_381,Allowance issued for the 2005-2007 period and ...
1,None,600500,2005,BE,EUA2005,GB_381,Allowance issued for the 2005-2007 period and ...
2,None,675008,2005,CZ,EUA2005,GB_381,Allowance issued for the 2005-2007 period and ...
3,None,983190,2005,DE,EUA2005,GB_381,Allowance issued for the 2005-2007 period and ...
4,None,224255,2005,DK,EUA2005,GB_381,Allowance issued for the 2005-2007 period and ...
5,None,895000,2005,EE,EUA2005,GB_381,Allowance issued for the 2005-2007 period and ...
6,None,5000,2005,ES,EUA2005,GB_381,Allowance issued for the 2005-2007 period and ...
7,None,237833,2005,FI,EUA2005,GB_381,Allowance issued for the 2005-2007 period and ...
8,None,485457,2005,FR,EUA2005,GB_381,Allowance issued for the 2005-2007 period and ...
9,None,16257826,2005,GB,EUA2005,GB_381,Allowance issued for the 2005-2007 period and ...


Up to 2012, the EUTL provides us with the type of unit surrendered as well as where these allowances have been originally created. Drax only used EU certificates either the ones released in the first or second period. If they would have used international credits such as CERs or ERUs, we should observe this in the data. After 2012, we only observe general allowances, i.e., EU allowances. 

Let's again zoom into the year 2005 and take a closer look at surrendered units:

In [13]:
df_surr_2005 = df_surrender[df_surrender.year == 2005]\
                    .drop(["year", "project_id", "unitType", "installation_id"], axis=1)
print("Surrender in 2005: ", df_surr_2005.amount.sum())
df_surr_2005

Surrender in 2005:  20771624


,amount,originatingRegistry_id,unitType_id
0,14000,AT,EUA2005
1,600500,BE,EUA2005
2,675008,CZ,EUA2005
3,983190,DE,EUA2005
4,224255,DK,EUA2005
5,895000,EE,EUA2005
6,5000,ES,EUA2005
7,237833,FI,EUA2005
8,485457,FR,EUA2005
9,16257826,GB,EUA2005


As a consistency check, the number of surrendered units in 2005 provided by the surrendering units details sums up to total surrendering as provided by compliance data.

Looking at the country of origin of surrendered units, we see a surprisingly diverse picture. 2005 was the first year of the EUETS and Drax needed to surrender units in April 2006. Although the instrument was rather new, Drax obtained allowances from various countries. This is particular surprising as Drax was rather generously equipped with allowances for free receiving about 14.5 million allowances for free in 2015 and also in 2016 (before surrendering). Staying at the descriptive level, we are not aiming to understand the drivers of Drax decision to make use of the allowance market but rather to understand how they managed to obtain these allowances in the very early phase of the EUETS.


# Accounts

The EUETS implements emissions trading. Industrial plants regulated are called <i>Installations</i>. Each installation has to be related to an accounts. These accounts might (and are likely to) change over time but at each instance in time an installation is represented by an account. Accounts are the backbone of the trading system, i.e., allowances are transferred between accounts not between installations. Accounts related to installations are called operator holding accounts. As transfers of allowances take place at the account level, we need to find out which accounts represent our Drax power station.

Technically, related accounts are again implemented as property on the installation object:

In [14]:
oha = drax.accounts
oha

[<Account(23963, 'GB Drax Power Station', 'GB', '100-7')>,
 <Account(22606, 'GB Operator Account', 'GB', '120-0')>]

There are two different accounts. 120-0 denotes former operator holding accounts whereas 100-7 denotes the operating holding account. This account structure is common for installations that have been regulated under the EUETS already before 2013. Due to the change in the EUTL, all installation had to convert their account from 120-0 to 100-7 accounts. Consequently, our 2005 analysis has to focus in the 120-0 account with the meaningful name "Operator Account".

Before analyzing the transactions of this account, let's see what kind of information we can obtain for the account: 

In [15]:
foha = oha[1]
foha.to_dict()

{'commitmentPeriod': 'Supplementary Program Commitment Period (2005 - 2007)',
 'openingDate': datetime.datetime(2005, 5, 18, 0, 0),
 'accountHolder_id': 10229,
 'accountType_id': '120-0',
 'name': 'GB Operator Account',
 'installation_id': 'GB_381',
 'companyRegistrationNumber': '4883589',
 'closingDate': datetime.datetime(2014, 7, 16, 16, 50, 14),
 'isOpen': False,
 'registry_id': 'GB',
 'id': 22606,
 'isRegisteredEutl': True,
 'accountType': 'Former Operator Holding Account'}

The account was opened in the Great Britain registry in May 2015 and closed after the revision of the EUTL in July 2014. The EUTL provides a company registration number useful to match accounts to other, firm-level datasets. The account provides a account holder reference pointing to the person responsible for the account:

In [17]:
foha.accountHolder.to_dict()

{'city': 'Selby',
 'addressSecondary': 'DRAX POWER STATION, PO BOX 3',
 'name': 'Drax Power Limited',
 'country_id': 'GB',
 'postalCode': 'YO8 8PH',
 'addressMain': 'Drax Power Station',
 'id': 10229,
 'country': 'United Kingdom'}

In the case of Drax, the account holder is not of particular interest. It is located at the same address and does not provide any further information. Often inspecting further accounts of the account holder can help disentangling the trading behavior:

In [20]:
foha.accountHolder.accounts

[<Account(23963, 'GB Drax Power Station', 'GB', '100-7')>,
 <Account(22606, 'GB Operator Account', 'GB', '120-0')>]

However, in this particular case we only see the current operator holding account which we already knew before. 

# Transactions

We now know the account Drax used to transfer allowances. We are, thus, finally in the position to analyze how Drax was able to surrender allowances from such a divers set of countries. To do so, we query transactions of the former operator holding account taking place before the date of surrendering (April 2006):

In [22]:
foha.transaction

AttributeError: 'Account' object has no attribute 'transaction'